In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

In [2]:
#Import data from csv
data=pd.read_csv('WeatherPy/city_data.csv')
#Find components for heat map
locations=data[['Lat','Long']]
Humidity=data['Humidity']
data.head()

,City Name,Lat,Long,Max Temp,Cloudiness,Humidity,Wind Speed,Country
0,kamenka,51.32,42.77,56.95,55.0,43.0,10.11,RU
1,tocopilla,-22.09,-70.20,66.31,21.0,53.0,11.34,CL
2,san cristobal,7.77,-72.22,91.40,40.0,43.0,20.80,VE
3,lebu,-37.62,-73.65,51.17,0.0,68.0,18.21,CL
4,dikson,73.51,80.55,45.66,69.0,90.0,19.95,RU


In [3]:
#Display heat map
gmaps.configure(api_key=g_key)
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [9]:
#Specify Data

#Not too windy
data_specific=data.loc[data['Wind Speed']<5]

#Not too hot
data_specific=data_specific.loc[data_specific['Max Temp']<90]

#Not too cold
data_specific=data_specific.loc[data_specific['Max Temp']>70]

#Not too cloudy
data_specific=data_specific.loc[data_specific['Cloudiness']<70]

#Not too humid
data_specific=data_specific.loc[data_specific['Humidity']<80]
data_specific=data_specific.reset_index(drop=True)
data_specific


,City Name,Lat,Long,Max Temp,Cloudiness,Humidity,Wind Speed,Country
0,cananea,30.95,-110.30,88.00,26.0,31.0,3.00,MX
1,ponta do sol,32.67,-17.10,71.60,20.0,78.0,3.36,PT
2,riyadh,24.69,46.72,87.80,10.0,25.0,3.27,SA
3,bulungu,-4.55,18.60,79.74,0.0,57.0,2.33,CD
4,castro,-24.79,-50.01,73.20,11.0,47.0,4.79,BR
5,dromolaxia,34.88,33.59,80.60,0.0,78.0,4.70,CY
6,bud,39.45,-86.18,80.01,40.0,57.0,4.70,US
7,kewanee,41.25,-89.92,77.00,0.0,57.0,1.99,US
8,catamarca,-28.47,-65.79,79.00,29.0,40.0,4.00,AR
9,salina,38.84,-97.61,82.99,1.0,37.0,4.70,US


In [10]:
#Initialize hotel dataframe

hotel_df=pd.DataFrame({'Hotel Name':[],
                       'Hotel Lat':[],
                       'Hotel Lng':[]})

#Run api search for hotels near each city
for x in range(len(data_specific)):
    #build url to search
    lat=data_specific['Lat'][x]
    lng=data_specific['Long'][x]
    target_coordinates=f'{lat},{lng}'
    target_radius=5000
    target_type='lodging'

    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #Exception in case no hotel in range (unlikely but possible)
    try:
        response = requests.get(base_url, params=params)
        hotel_data = response.json()

        hotel_df.at[x,'Hotel Name']=hotel_data['results'][0]['name']
        hotel_df.at[x,'Hotel Lat']=hotel_data['results'][0]['geometry']['location']['lat']
        hotel_df.at[x,'Hotel Lng']=hotel_data['results'][0]['geometry']['location']['lng']
    except:
        hotel_df.at[x,'Hotel Name']='No Hotel Found'

#Remove rows if no hotel found
hotel_df=hotel_df.loc[hotel_df['Hotel Lat'].isnull()==False]
hotel_df.head()

,Hotel Name,Hotel Lat,Hotel Lng
0,Casas Montoya,30.984748,-110.303769
1,Hotel do Campo,32.676959,-17.066139
2,InterContinental Riyadh,24.666677,46.692159
4,CHACARA BAILLY,-24.780711,-49.986828
5,Flamingo Beach Hotel,34.893684,33.638300


In [11]:
# Add marker layer ontop of heat map
fig = gmaps.figure(layout=figure_layout)
locations_specific=data_specific[['Lat','Long']]
Humidity_specific=data_specific['Humidity']
heat_layer = gmaps.heatmap_layer(locations_specific, weights=Humidity_specific, 
                                 dissipating=False, max_intensity=max(Humidity_specific),
                                 point_radius=1)
fig.add_layer(heat_layer)
markers_coord=hotel_df[['Hotel Lat', 'Hotel Lng']]
markers=gmaps.marker_layer(markers_coord)
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…